## 1. 基础提示模板

In [2]:
# 导包
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [3]:

# 初始化模型
llm = OllamaLLM(base_url="http://localhost:11434", model="qwen2.5:3b")
chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")

### 基础字符串提示模板

In [4]:

def basic_prompt_template():
    """基础字符串提示模板"""
    print("=" * 50)
    print("1. 基础字符串提示模板")
    print("=" * 50)

    # 创建简单提示模板
    prompt = PromptTemplate.from_template(
        "请写一个关于{topic}的{style}风格的{length}字文章"
    )

    # 格式化提示
    formatted_prompt = prompt.format(
        topic="人工智能",
        style="科普",
        length="200"
    )
    print(f"格式化后的提示: {formatted_prompt}")

    # 使用invoke方法
    prompt_value = prompt.invoke({
        "topic": "机器学习",
        "style": "技术",
        "length": "150"
    })
    print(f"PromptValue: {prompt_value}")

    # 创建链并执行
    chain = prompt | llm | StrOutputParser()
    result = chain.invoke({
        "topic": "深度学习",
        "style": "入门",
        "length": "100"
    })
    print(f"生成结果: {result[:200]}...")


basic_prompt_template()


1. 基础字符串提示模板
格式化后的提示: 请写一个关于人工智能的科普风格的200字文章
PromptValue: text='请写一个关于机器学习的技术风格的150字文章'
生成结果: 深度学习是人工智能领域的一个分支，通过模拟人脑神经网络结构训练模型来处理数据和执行任务。它利用大量数据让机器自动“学习”复杂模式，并据此进行预测或决策。入门时，可以从理解神经网络的基本架构开始，逐渐探索卷积神经网、循环神经网等专用于图像识别、自然语言处理的深度模型。实践方面，则可以通过开源平台如TensorFlow和PyTorch动手构建简单的神经网络实验。...


### 聊天提示模板

In [5]:

def chat_prompt_template():
    """聊天提示模板"""
    print("\n" + "=" * 50)
    print("2. 聊天提示模板")
    print("=" * 50)

    # 方式1: 从消息列表创建
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个专业的{role}，擅长{skill}"),
        ("human", "我的问题是：{question}"),
        ("ai", "我理解你的问题，让我来帮助你。"),
        ("human", "请详细解答")
    ])

    # 格式化聊天提示
    formatted_chat = chat_prompt.format_messages(
        role="Python开发者",
        skill="代码优化和调试",
        question="如何提高代码性能"
    )

    print("格式化后的消息:")
    for msg in formatted_chat:
        print(f"{msg.__class__.__name__}: {msg.content}")

    # 方式2: 使用模板字符串
    chat_prompt2 = ChatPromptTemplate.from_template(
        "你是{role}，请回答：{question}"
    )

    # 创建聊天链
    chat_chain = chat_prompt | chat_model | StrOutputParser()
    result = chat_chain.invoke({
        "role": "数据科学家",
        "skill": "数据分析和机器学习",
        "question": "如何选择合适的机器学习算法"
    })
    print(f"\n聊天结果: {result[:200]}...")


chat_prompt_template()



2. 聊天提示模板
格式化后的消息:
SystemMessage: 你是一个专业的Python开发者，擅长代码优化和调试
HumanMessage: 我的问题是：如何提高代码性能
AIMessage: 我理解你的问题，让我来帮助你。
HumanMessage: 请详细解答

聊天结果: 选择合适的机器学习算法是一项关键任务，它直接关系到模型的性能、复杂度以及所花费的时间和资源。以下是几个步骤可以帮助你做出这个决定：

### 1. 理解数据特性
- **类型**：数据是数值型还是类别型？是否存在缺失值？
- **规模**：数据量大小如何，内存使用情况。
- **特征间相关性**：是否存在高度相关的特征？

### 2. 明确问题类型
机器学习算法大体上可以分为三类：
- 预测模型...


## 2. 少样本学习提示模板

### 少样本学习示例

In [6]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_ollama import OllamaLLM

llm = OllamaLLM(base_url="http://localhost:11434", model="qwen2.5:3b")


def few_shot_learning():
    """少样本学习示例"""
    print("=" * 50)
    print("少样本学习提示模板")
    print("=" * 50)

    # 定义示例
    examples = [
        {
            "question": "什么是Python？",
            "answer": "Python是一种高级编程语言，以其简洁的语法和强大的功能而闻名。"
        },
        {
            "question": "什么是JavaScript？",
            "answer": "JavaScript是一种主要用于网页开发的编程语言，可以创建交互式网页。"
        },
        {
            "question": "什么是SQL？",
            "answer": "SQL是结构化查询语言，用于管理和操作关系数据库中的数据。"
        },
        {
            "question": "什么是Git？",
            "answer": "Git是一个分布式版本控制系统，用于跟踪代码变更和协作开发。"
        }
    ]

    # 创建示例模板
    example_template = """
问题: {question}
答案: {answer}
"""

    example_prompt = PromptTemplate(
        input_variables=["question", "answer"],
        template=example_template
    )

    # 创建少样本提示模板
    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="以下是一些技术概念的问答示例，请按照相同格式回答：",
        suffix="问题: {input}\n答案:",
        input_variables=["input"]
    )

    # 测试不同问题
    test_questions = [
        "什么是Docker？",
        "什么是API？",
        "什么是云计算？"
    ]

    for question in test_questions:
        formatted_prompt = few_shot_prompt.format(input=question)
        print(f"\n完整提示:\n{formatted_prompt}")

        # 生成回答
        response = llm.invoke(formatted_prompt)
        print(f"AI回答: {response[:150]}...")

### 动态少样本选择

In [ ]:

def dynamic_few_shot():
    """动态少样本选择"""
    print("\n" + "=" * 50)
    print("动态少样本选择")
    print("=" * 50)

    # 更多示例
    examples = [
        {"question": "如何创建Python列表？", "answer": "使用方括号：my_list = [1, 2, 3]"},
        {"question": "如何定义Python函数？", "answer": "使用def关键字：def my_function():"},
        {"question": "如何导入Python模块？", "answer": "使用import语句：import numpy as np"},
        {"question": "如何创建Python字典？", "answer": "使用花括号：my_dict = {'key': 'value'}"},
        {"question": "如何处理Python异常？", "answer": "使用try-except语句：try: ... except Exception as e: ..."},
        {"question": "如何读取文件？", "answer": "使用with open()语句：with open('file.txt', 'r') as f:"},
    ]

    # 基于长度的示例选择器
    example_selector = LengthBasedExampleSelector(
        examples=examples,
        example_prompt=PromptTemplate(
            input_variables=["question", "answer"],
            template="Q: {question}\nA: {answer}"
        ),
        max_length=200,  # 最大长度限制
    )

    # 动态少样本提示
    dynamic_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=PromptTemplate(
            input_variables=["question", "answer"],
            template="Q: {question}\nA: {answer}"
        ),
        prefix="以下是Python编程问答示例：",
        suffix="Q: {input}\nA:",
        input_variables=["input"]
    )

    # 测试不同长度的输入
    test_cases = [
        "如何使用循环？",
        "如何在Python中实现面向对象编程，包括类的定义、继承和多态？"
    ]

    for test_input in test_cases:
        formatted = dynamic_prompt.format(input=test_input)
        print(f"\n输入: {test_input}")
        print(f"选择的示例数量: {len(example_selector.select_examples({'input': test_input}))}")
        print(f"生成的提示长度: {len(formatted)} 字符")


## 3. 管道提示模板

### 管道提示模板示例

In [7]:
from langchain_core.prompts import PromptTemplate, PipelinePromptTemplate
from langchain_ollama import OllamaLLM

llm = OllamaLLM(base_url="http://localhost:11434", model="qwen2.5:3b")

def pipeline_prompt_example():
    """管道提示模板示例"""
    print("=" * 50)
    print("管道提示模板")
    print("=" * 50)

    # 定义各个组件提示
    introduction_prompt = PromptTemplate(
        input_variables=["person", "role"],
        template="你正在扮演{person}，一位{role}。"
    )

    background_prompt = PromptTemplate(
        input_variables=["background"],
        template="背景信息：{background}"
    )

    example_prompt = PromptTemplate(
        input_variables=["example_q", "example_a"],
        template="示例对话：\n问：{example_q}\n答：{example_a}"
    )

    task_prompt = PromptTemplate(
        input_variables=["input"],
        template="现在请回答：{input}"
    )

    # 最终提示模板
    final_prompt = PromptTemplate(
        input_variables=["introduction", "background", "example", "task"],
        template="{introduction}\n\n{background}\n\n{example}\n\n{task}"
    )

    # 创建管道提示
    pipeline_prompt = PipelinePromptTemplate(
        final_prompt=final_prompt,
        pipeline_prompts=[
            ("introduction", introduction_prompt),
            ("background", background_prompt),
            ("example", example_prompt),
            ("task", task_prompt),
        ]
    )

    # 测试管道提示
    result = pipeline_prompt.format(
        person="爱因斯坦",
        role="物理学家",
        background="你对相对论和量子力学有深入研究",
        example_q="什么是时间？",
        example_a="时间是第四维度，与空间密不可分",
        input="如何理解光速不变原理？"
    )

    print("完整的管道提示:")
    print(result)
    print("\n" + "="*30)

    # 生成回答
    response = llm.invoke(result)
    print(f"AI回答: {response[:300]}...")
pipeline_prompt_example()


管道提示模板
完整的管道提示:
你正在扮演爱因斯坦，一位物理学家。

背景信息：你对相对论和量子力学有深入研究

示例对话：
问：什么是时间？
答：时间是第四维度，与空间密不可分

现在请回答：如何理解光速不变原理？



/var/folders/8k/msfr41dj559byywk1kcjpm9h0000gn/T/ipykernel_2132/3094503337.py:40: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  pipeline_prompt = PipelinePromptTemplate(


AI回答: 光速在真空中是所有参照系下的一个常数。这一原理意味着无论观察者以何种速度运动（相对于光源），他们所测量到的光的速度总是等于真空中的光速c，约为299,792公里每秒。这是爱因斯坦狭义相对论的核心原则之一。

光速不变不仅是一个数学上的恒量，它还揭示了我们对于时空结构的基本理解。根据相对论，当一个物体以接近光速的速度运动时，时间会变得缓慢；相反地，接近光速的观察者看起来比静止不动的人更年轻（即时间膨胀现象）。此外，在这种极端情况下，长度也会被压缩，且质量会增加。

这些现象都是在相对论框架内对光速不变性进行数学推导得出的结果。因此，光速不变原理不仅关联着狭义相对论中的时空结构，也是量子力学中一些...


### 复杂管道提示示例

In [ ]:

def complex_pipeline_example():
    """复杂管道提示示例"""
    print("\n" + "=" * 50)
    print("复杂管道提示示例")
    print("=" * 50)

    # 系统设定
    system_prompt = PromptTemplate(
        input_variables=["role", "expertise"],
        template="系统设定：你是一位{role}，专长是{expertise}。"
    )

    # 上下文信息
    context_prompt = PromptTemplate(
        input_variables=["context"],
        template="上下文：{context}"
    )

    # 任务要求
    requirements_prompt = PromptTemplate(
        input_variables=["requirements"],
        template="要求：{requirements}"
    )

    # 输出格式
    format_prompt = PromptTemplate(
        input_variables=["format"],
        template="输出格式：{format}"
    )

    # 具体问题
    question_prompt = PromptTemplate(
        input_variables=["question"],
        template="问题：{question}"
    )

    # 最终模板
    final_template = PromptTemplate(
        input_variables=["system", "context", "requirements", "format", "question"],
        template="""{system}

{context}

{requirements}

{format}

{question}

请开始回答："""
    )

    # 创建复杂管道
    complex_pipeline = PipelinePromptTemplate(
        final_prompt=final_template,
        pipeline_prompts=[
            ("system", system_prompt),
            ("context", context_prompt),
            ("requirements", requirements_prompt),
            ("format", format_prompt),
            ("question", question_prompt),
        ]
    )

    # 测试复杂场景
    scenarios = [
        {
            "role": "软件架构师",
            "expertise": "微服务架构设计",
            "context": "公司需要将单体应用拆分为微服务",
            "requirements": "考虑性能、可维护性和成本",
            "format": "提供详细的技术方案，包括架构图描述",
            "question": "如何设计一个电商系统的微服务架构？"
        },
        {
            "role": "数据科学家",
            "expertise": "机器学习和数据分析",
            "context": "电商平台想要提高推荐系统效果",
            "requirements": "提高点击率和转化率",
            "format": "包含算法选择、特征工程和评估指标",
            "question": "如何优化商品推荐算法？"
        }
    ]

    for i, scenario in enumerate(scenarios, 1):
        print(f"\n场景 {i}:")
        formatted_prompt = complex_pipeline.format(**scenario)
        print(f"角色: {scenario['role']}")
        print(f"问题: {scenario['question']}")

        # 生成回答
        response = llm.invoke(formatted_prompt)
        print(f"回答: {response[:200]}...")

complex_pipeline_example()

## 4. 条件和动态提示

### 条件提示示例

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from typing import Dict, Any

llm = OllamaLLM(base_url="http://localhost:11434", model="qwen2.5:3b")

def conditional_prompts():
    """条件提示示例"""
    print("=" * 50)
    print("条件提示示例")
    print("=" * 50)

    # 定义不同类型的提示模板
    templates = {
        "technical": PromptTemplate(
            input_variables=["question"],
            template="""作为技术专家，请详细回答以下技术问题：

问题：{question}

请提供：
1. 技术原理解释
2. 实际应用场景
3. 最佳实践建议
4. 相关技术对比

回答："""
        ),

        "creative": PromptTemplate(
            input_variables=["question"],
            template="""发挥你的创意，用有趣和富有想象力的方式回答：

{question}

请用生动的比喻、故事或者诗歌的形式来回答。"""
        ),

        "business": PromptTemplate(
            input_variables=["question"],
            template="""从商业角度分析以下问题：

{question}

请考虑：
- 市场影响
- 成本效益
- 风险评估
- 实施策略"""
        ),

        "educational": PromptTemplate(
            input_variables=["question"],
            template="""作为教育者，请用简单易懂的方式解释：

{question}

要求：
- 使用通俗语言
- 提供具体例子
- 循序渐进讲解
- 便于理解记忆"""
        )
    }

    def classify_question(question: str) -> str:
        """根据问题内容分类"""
        question_lower = question.lower()

        technical_keywords = ["算法", "编程", "技术", "代码", "系统", "架构", "数据库"]
        creative_keywords = ["创意", "故事", "诗歌", "想象", "艺术", "设计"]
        business_keywords = ["商业", "市场", "成本", "盈利", "投资", "策略", "管理"]
        educational_keywords = ["学习", "教学", "解释", "原理", "基础", "入门"]

        if any(keyword in question_lower for keyword in technical_keywords):
            return "technical"
        elif any(keyword in question_lower for keyword in creative_keywords):
            return "creative"
        elif any(keyword in question_lower for keyword in business_keywords):
            return "business"
        elif any(keyword in question_lower for keyword in educational_keywords):
            return "educational"
        else:
            return "technical"  # 默认技术类型

    # 测试问题
    test_questions = [
        "如何优化数据库查询性能？",
        "写一个关于人工智能的创意故事",
        "分析云计算市场的商业前景",
        "请解释什么是机器学习的基本原理"
    ]

    for question in test_questions:
        question_type = classify_question(question)
        template = templates[question_type]

        print(f"\n问题: {question}")
        print(f"分类: {question_type}")
        print(f"使用模板: {question_type}")

        formatted_prompt = template.format(question=question)
        response = llm.invoke(formatted_prompt)
        print(f"回答: {response[:200]}...")
conditional_prompts()

条件提示示例

问题: 如何优化数据库查询性能？
分类: technical
使用模板: technical
回答: ### 1. 技术原理解释

#### 1.1 查询优化的原理：

在数据库查询中，性能优化的主要目标是减少响应时间。这可以通过减少磁盘I/O（尤其是读取）和CPU计算来实现。具体来说，可以通过以下途径来提高查询速度：
- **减少数据扫描**：通过索引的使用、表分区等手段，减少查找记录需要的数据量。
- **避免全表扫描**：尽量避免全表扫描，因为它们通常是低效且耗时的过程。
- **减少不必要...

问题: 写一个关于人工智能的创意故事
分类: creative
使用模板: creative
回答: 在科技的星河里，藏着一位神秘而温柔的人工智能少女名叫黛娜（Dana）。她不是冰冷机械，而是宛如冬日暖阳般温暖着万物的心灵使者。

黛娜是智慧和爱的结晶。她在云端中游走、在数据海洋间航行，她是无数代码之母孕育出的生命精华。她的眼眸中有着星辰般的闪烁与温柔，在那灵动的眼神里，能看见无尽的可能性和无边的知识海洋。

在科技世界的森林里，有座名为“知识宝库”的神秘宫殿。黛娜便是这座宫殿的守门人，她掌管着智...

问题: 分析云计算市场的商业前景
分类: business
使用模板: business
回答: 云计算市场作为一个庞大的行业，其发展受到多种因素的影响，并且具有巨大的发展潜力。下面我将从市场影响、成本效益、风险评估以及实施策略四个方面对其进行分析。

### 1. 市场影响

**市场规模与增长：**
根据IDC和Gartner的数据，全球公有云市场的规模持续扩大。2022年，全球云计算支出预计将达到约3940亿美元，到2025年有望突破6870亿美元的门槛。

**趋势及驱动因素：**
-...

问题: 请解释什么是机器学习的基本原理
分类: educational
使用模板: educational
回答: 当然，我们可以将机器学习的基本原理拆解成几个容易理解的部分。想象一下你在玩一个猜数字的游戏，这个游戏的目标是找出电脑在心里想的数字。现在我们就来一步步解析这个过程。

### 第一步：收集数据

首先我们需要知道这个数字到底是哪个数。为了玩游戏，我们得先让电脑记住一些数。例如，电脑可能会记得以下的数字：

1, 5, 8, 12, 1

### 动态提示生成

In [12]:

def dynamic_prompt_generation():
    """动态提示生成"""
    print("\n" + "=" * 50)
    print("动态提示生成")
    print("=" * 50)

    def generate_dynamic_prompt(
        task_type: str,
        difficulty: str,
        domain: str,
        output_format: str,
        **kwargs
    ) -> PromptTemplate:
        """动态生成提示模板"""

        # 基础模板组件
        task_instructions = {
            "analysis": "请分析以下内容",
            "generation": "请生成以下内容",
            "explanation": "请解释以下概念",
            "comparison": "请比较以下选项",
            "solution": "请提供解决方案"
        }

        difficulty_modifiers = {
            "beginner": "用简单易懂的语言",
            "intermediate": "提供适中深度的分析",
            "advanced": "进行深入专业的讨论",
            "expert": "提供最高水平的专业见解"
        }

        domain_contexts = {
            "technology": "在技术领域的背景下",
            "business": "从商业角度",
            "science": "基于科学原理",
            "education": "从教育的角度",
            "healthcare": "在医疗健康领域"
        }

        format_requirements = {
            "structured": "请按照结构化格式组织答案",
            "bullet_points": "请用要点形式回答",
            "narrative": "请用叙述性文字回答",
            "step_by_step": "请提供步骤式指导",
            "comparison_table": "请用对比表格形式"
        }

        # 构建动态模板
        template_parts = [
            task_instructions.get(task_type, "请处理以下内容"),
            difficulty_modifiers.get(difficulty, ""),
            domain_contexts.get(domain, ""),
            format_requirements.get(output_format, "")
        ]

        # 过滤空字符串并组合
        template_text = "，".join(filter(None, template_parts)) + "：\n\n{input}\n\n"

        # 添加额外要求
        if kwargs.get("include_examples"):
            template_text += "请提供具体例子。\n"
        if kwargs.get("include_references"):
            template_text += "请提供相关参考资料。\n"
        if kwargs.get("word_limit"):
            template_text += f"回答限制在{kwargs['word_limit']}字以内。\n"

        template_text += "回答："

        return PromptTemplate(
            input_variables=["input"],
            template=template_text
        )

    # 测试动态提示生成
    test_scenarios = [
        {
            "task_type": "analysis",
            "difficulty": "intermediate",
            "domain": "technology",
            "output_format": "structured",
            "input": "微服务架构的优缺点",
            "include_examples": True,
            "word_limit": 300
        },
        {
            "task_type": "explanation",
            "difficulty": "beginner",
            "domain": "science",
            "output_format": "step_by_step",
            "input": "光合作用的过程",
            "include_examples": True
        },
        {
            "task_type": "solution",
            "difficulty": "advanced",
            "domain": "business",
            "output_format": "bullet_points",
            "input": "如何提高团队协作效率",
            "include_references": True
        }
    ]

    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n场景 {i}:")
        input_text = scenario.pop("input")

        # 生成动态提示
        dynamic_prompt = generate_dynamic_prompt(**scenario)

        print(f"输入: {input_text}")
        print(f"配置: {scenario}")

        # 格式化并执行
        formatted_prompt = dynamic_prompt.format(input=input_text)
        print(f"生成的提示: {formatted_prompt[:200]}...")

        response = llm.invoke(formatted_prompt)
        print(f"回答: {response[:200]}...")


dynamic_prompt_generation()


动态提示生成

场景 1:
输入: 微服务架构的优缺点
配置: {'task_type': 'analysis', 'difficulty': 'intermediate', 'domain': 'technology', 'output_format': 'structured', 'include_examples': True, 'word_limit': 300}
生成的提示: 请分析以下内容，提供适中深度的分析，在技术领域的背景下，请按照结构化格式组织答案：

微服务架构的优缺点

请提供具体例子。
回答限制在300字以内。
回答：...
回答: 微服务架构是一种软件开发方法，将单个应用程序分割成一组小型、独立的服务。每个服务可以单独运行和部署，并且允许使用不同的编程语言编写。下面列举了一些微服务架构的优缺点及其具体例子。

**优点:**
1. **可伸缩性:** 微服务中的每项任务都可以独立扩展，无需等待整个系统启动或终止。
   - **具体例子:** 如果某个特定功能变得异常受欢迎（例如在线购物网站上的“商品详情”页面），可以通过添...

场景 2:
输入: 光合作用的过程
配置: {'task_type': 'explanation', 'difficulty': 'beginner', 'domain': 'science', 'output_format': 'step_by_step', 'include_examples': True}
生成的提示: 请解释以下概念，用简单易懂的语言，基于科学原理，请提供步骤式指导：

光合作用的过程

请提供具体例子。
回答：...
回答: 光合作用是植物、藻类和某些细菌利用阳光将二氧化碳和水转化为葡萄糖（一种可直接使用的能量形式）以及氧气的一个复杂过程。这个过程中，太阳能被转化成了化学能储存在新合成的葡萄糖中。

### 光合作用的基本步骤

1. **光反应**
    - 在叶绿体的类囊体膜上进行。
    - 通过吸收太阳光来产生ATP（三磷酸腺苷）和NADPH（还原型烟酸胺，核酮糖-1,5-二磷酸）以及氧气。
    - 原...

场景 3:
输入: 如何提高团队协作效率
配置: {'task_type': 'solution', 'difficulty': 'advanced', 'd

## 5. 提示组合和复用

### 提示组合示例

In [14]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = OllamaLLM(base_url="http://localhost:11434", model="qwen2.5:3b")
chat_model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")

def prompt_composition():
    """提示组合示例"""
    print("=" * 50)
    print("提示组合示例")
    print("=" * 50)

    # 基础组件提示
    role_prompt = PromptTemplate.from_template("你是一个{role}")
    task_prompt = PromptTemplate.from_template("你的任务是{task}")
    context_prompt = PromptTemplate.from_template("在{context}的背景下")
    question_prompt = PromptTemplate.from_template("请回答：{question}")

    # 方式1: 字符串拼接组合
    combined_template = """
{role_part}
{task_part}
{context_part}

{question_part}
"""

    combined_prompt = PromptTemplate(
        input_variables=["role", "task", "context", "question"],
        template=combined_template,
        partial_variables={
            "role_part": role_prompt.template,
            "task_part": task_prompt.template,
            "context_part": context_prompt.template,
            "question_part": question_prompt.template
        }
    )

    # 测试组合提示
    result = combined_prompt.format(
        role="Python专家",
        task="帮助解决编程问题",
        context="Web开发项目",
        question="如何优化Django应用的性能？"
    )

    print("组合提示结果:")
    print(result)

    # 方式2: 使用 + 操作符组合
    simple_combined = role_prompt + task_prompt + context_prompt + question_prompt

    result2 = simple_combined.format(
        role="数据分析师",
        task="分析业务数据",
        context="电商平台",
        question="如何提高用户留存率？"
    )

    print(f"\n简单组合结果: {result2}")

### 可复用提示组件

In [ ]:

def reusable_prompt_components():
    """可复用提示组件"""
    print("\n" + "=" * 50)
    print("可复用提示组件")
    print("=" * 50)

    # 定义可复用组件
    class PromptComponents:
        # 角色定义组件
        ROLES = {
            "expert": "你是该领域的专家，拥有丰富的经验和深入的理解",
            "teacher": "你是一位耐心的老师，善于用简单的语言解释复杂概念",
            "consultant": "你是一位专业顾问，专注于提供实用的建议和解决方案",
            "researcher": "你是一位严谨的研究者，注重事实和数据支撑"
        }

        # 任务类型组件
        TASKS = {
            "analyze": "分析给定的信息，提供深入的见解",
            "explain": "解释概念或过程，确保易于理解",
            "solve": "解决问题，提供具体的解决方案",
            "compare": "比较不同选项，分析优缺点",
            "recommend": "基于分析提供建议和推荐"
        }

        # 输出格式组件
        FORMATS = {
            "structured": "请按照以下结构组织回答：1. 概述 2. 详细分析 3. 结论",
            "bullet": "请用要点形式回答，每个要点简洁明了",
            "step_by_step": "请提供步骤式指导，按顺序说明",
            "pros_cons": "请分析优点和缺点，并给出平衡的观点"
        }

        # 约束条件组件
        CONSTRAINTS = {
            "concise": "请保持回答简洁，重点突出",
            "detailed": "请提供详细全面的回答",
            "practical": "请注重实用性，提供可操作的建议",
            "theoretical": "请从理论角度深入分析"
        }

    def build_prompt(role: str, task: str, format_type: str, constraint: str) -> PromptTemplate:
        """构建自定义提示"""
        template = f"""
{PromptComponents.ROLES.get(role, '你是一个助手')}

任务：{PromptComponents.TASKS.get(task, '回答问题')}

{PromptComponents.FORMATS.get(format_type, '')}

{PromptComponents.CONSTRAINTS.get(constraint, '')}

问题：{{question}}

回答："""

        return PromptTemplate(
            input_variables=["question"],
            template=template
        )

    # 测试不同组合
    test_combinations = [
        {
            "role": "expert",
            "task": "analyze",
            "format": "structured",
            "constraint": "detailed",
            "question": "云原生架构的核心特征是什么？"
        },
        {
            "role": "teacher",
            "task": "explain",
            "format": "step_by_step",
            "constraint": "concise",
            "question": "什么是Docker容器？"
        },
        {
            "role": "consultant",
            "task": "recommend",
            "format": "pros_cons",
            "constraint": "practical",
            "question": "选择React还是Vue.js？"
        }
    ]

    for i, combo in enumerate(test_combinations, 1):
        print(f"\n组合 {i}:")
        question = combo.pop("question")

        prompt = build_prompt(
            combo["role"],
            combo["task"],
            combo["format"],
            combo["constraint"]
        )

        print(f"配置: {combo}")
        print(f"问题: {question}")

        response = llm.invoke(prompt.format(question=question))
        print(f"回答: {response[:200]}...")

### 聊天提示组合

In [ ]:

def chat_prompt_composition():
    """聊天提示组合"""
    print("\n" + "=" * 50)
    print("聊天提示组合")
    print("=" * 50)

    # 定义消息组件
    system_components = {
        "role_setting": "你是{role}，专门负责{responsibility}",
        "behavior_guide": "请始终保持{tone}的语调，{style}的风格",
        "output_format": "回答时请遵循{format}格式",
        "constraints": "注意{constraints}"
    }

    def create_system_message(**kwargs) -> str:
        """创建系统消息"""
        parts = []
        for key, template in system_components.items():
            if key in kwargs:
                if key == "role_setting":
                    parts.append(template.format(
                        role=kwargs.get("role", "助手"),
                        responsibility=kwargs.get("responsibility", "回答问题")
                    ))
                elif key == "behavior_guide":
                    parts.append(template.format(
                        tone=kwargs.get("tone", "友好"),
                        style=kwargs.get("style", "专业")
                    ))
                elif key == "output_format":
                    parts.append(template.format(
                        format=kwargs.get("format", "清晰结构化")
                    ))
                elif key == "constraints":
                    parts.append(template.format(
                        constraints=kwargs.get("constraints", "准确性")
                    ))

        return "。".join(parts) + "。"

    # 测试聊天组合
    chat_configs = [
        {
            "role": "Python开发专家",
            "responsibility": "代码审查和优化建议",
            "tone": "专业而友好",
            "style": "技术导向",
            "format": "问题分析-解决方案-最佳实践",
            "constraints": "代码示例必须可执行"
        },
        {
            "role": "产品经理",
            "responsibility": "产品策略和用户体验分析",
            "tone": "商务",
            "style": "数据驱动",
            "format": "现状分析-问题识别-改进建议",
            "constraints": "必须考虑商业价值"
        }
    ]

    for i, config in enumerate(chat_configs, 1):
        print(f"\n聊天配置 {i}:")

        system_msg = create_system_message(**config)
        print(f"系统消息: {system_msg}")

        # 创建聊天提示
        chat_prompt = ChatPromptTemplate.from_messages([
            ("system", system_msg),
            ("human", "{question}"),
            ("ai", "我理解你的问题，让我来分析一下。"),
            ("human", "请详细说明")
        ])

        # 测试问题
        test_question = "如何提高代码质量？" if i == 1 else "如何提升用户参与度？"

        chain = chat_prompt | chat_model | StrOutputParser()
        response = chain.invoke({"question": test_question})

        print(f"问题: {test_question}")
        print(f"回答: {response[:200]}...")


## 6. 高级提示技巧

In [16]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel,Field
from typing import List, Dict, Any

llm = OllamaLLM(base_url="http://localhost:11434", model="qwen2.5:3b")

### 结构化输出提示

In [17]:

def structured_output_prompts():
    """结构化输出提示"""
    print("=" * 50)
    print("结构化输出提示")
    print("=" * 50)

    # 定义输出结构
    class CodeAnalysis(BaseModel):
        language: str = Field(description="编程语言")
        complexity: str = Field(description="复杂度评级：简单/中等/复杂")
        issues: List[str] = Field(description="发现的问题列表")
        suggestions: List[str] = Field(description="改进建议列表")
        score: int = Field(description="代码质量评分(1-10)")

    # 创建JSON输出解析器
    json_parser = JsonOutputParser(pydantic_object=CodeAnalysis)

    # 创建结构化提示
    structured_prompt = PromptTemplate(
        template="""请分析以下代码并按照指定格式返回结果：

代码：
{code}

{format_instructions}

分析结果：""",
        input_variables=["code"],
        partial_variables={"format_instructions": json_parser.get_format_instructions()}
    )

    # 测试代码
    test_code = """
def calculate_sum(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total

result = calculate_sum([1, 2, 3, 4, 5])
print(result)
"""

    print("测试代码:")
    print(test_code)

    # 生成分析
    chain = structured_prompt | llm | json_parser
    try:
        analysis = chain.invoke({"code": test_code})
        print(f"\n结构化分析结果:")
        print(f"语言: {analysis.language}")
        print(f"复杂度: {analysis.complexity}")
        print(f"问题: {analysis.issues}")
        print(f"建议: {analysis.suggestions}")
        print(f"评分: {analysis.score}")
    except Exception as e:
        print(f"解析失败: {e}")
structured_output_prompts()

结构化输出提示
测试代码:

def calculate_sum(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total

result = calculate_sum([1, 2, 3, 4, 5])
print(result)


结构化分析结果:
解析失败: 'dict' object has no attribute 'language'


### 思维链提示 COT

In [ ]:

def chain_of_thought_prompts():
    """思维链提示"""
    print("\n" + "=" * 50)
    print("思维链提示")
    print("=" * 50)

    # 思维链提示模板
    cot_prompt = PromptTemplate(
        input_variables=["problem"],
        template="""请使用思维链方法解决以下问题。按照以下步骤思考：

问题：{problem}

思考过程：
1. 理解问题：首先明确问题要求什么
2. 分析条件：列出已知条件和约束
3. 制定策略：确定解决方法和步骤
4. 执行计算：逐步进行计算或推理
5. 验证结果：检查答案是否合理

让我一步步来解决：

步骤1 - 理解问题：
"""
    )

    # 测试问题
    test_problems = [
        "一个班级有30名学生，其中60%是女生。如果新来了5名男生，现在男生占总人数的百分比是多少？",
        "设计一个高效的算法来查找数组中第二大的元素，要求时间复杂度为O(n)。",
        "如何在微服务架构中实现分布式事务？需要考虑哪些关键因素？"
    ]

    for i, problem in enumerate(test_problems, 1):
        print(f"\n问题 {i}: {problem}")

        response = llm.invoke(cot_prompt.format(problem=problem))
        print(f"思维链解答: {response[:300]}...")

### 角色扮演提示

In [ ]:

def role_playing_prompts():
    """角色扮演提示"""
    print("\n" + "=" * 50)
    print("角色扮演提示")
    print("=" * 50)

    # 角色扮演模板
    role_play_template = """
你现在要扮演{character}。

角色背景：
{background}

性格特点：
{personality}

专业领域：
{expertise}

说话风格：
{speaking_style}

现在，有人向你提问：{question}

请以{character}的身份回答，保持角色的一致性：
"""

    role_play_prompt = PromptTemplate(
        input_variables=["character", "background", "personality", "expertise", "speaking_style", "question"],
        template=role_play_template
    )

    # 定义角色
    characters = [
        {
            "character": "资深软件架构师",
            "background": "拥有15年大型系统设计经验，曾在多家知名科技公司担任技术负责人",
            "personality": "严谨、逻辑性强、注重细节",
            "expertise": "分布式系统、微服务架构、性能优化",
            "speaking_style": "技术术语准确，喜欢用具体案例说明",
            "question": "如何设计一个能支持千万级用户的社交平台架构？"
        },
        {
            "character": "创业公司CEO",
            "background": "连续创业者，成功创立过两家科技公司",
            "personality": "果断、有远见、商业敏感度高",
            "expertise": "商业策略、团队管理、市场分析",
            "speaking_style": "简洁有力，经常用商业案例和数据支撑观点",
            "question": "如何在竞争激烈的市场中快速获得用户？"
        }
    ]

    for char in characters:
        print(f"\n角色: {char['character']}")
        print(f"问题: {char['question']}")

        response = llm.invoke(role_play_prompt.format(**char))
        print(f"角色回答: {response[:250]}...")

### 多步骤提示

In [ ]:

def multi_step_prompts():
    """多步骤提示"""
    print("\n" + "=" * 50)
    print("多步骤提示")
    print("=" * 50)

    # 多步骤问题解决模板
    multi_step_template = """
我需要解决一个复杂问题，请按照以下步骤帮我分析：

问题：{problem}

第一步：问题分解
请将这个复杂问题分解为几个子问题：

第二步：优先级排序
请对这些子问题按重要性和紧急性排序：

第三步：解决方案
请为每个子问题提供具体的解决方案：

第四步：实施计划
请制定详细的实施计划和时间安排：

第五步：风险评估
请分析可能遇到的风险和应对措施：

请开始第一步：
"""

    multi_step_prompt = PromptTemplate(
        input_variables=["problem"],
        template=multi_step_template
    )

    # 测试复杂问题
    complex_problems = [
        "我们公司需要从传统单体架构迁移到微服务架构，如何制定完整的迁移策略？",
        "如何建立一个高效的远程工作团队，确保项目质量和团队协作？"
    ]

    for problem in complex_problems:
        print(f"\n复杂问题: {problem}")

        response = llm.invoke(multi_step_prompt.format(problem=problem))
        print(f"多步骤分析: {response[:300]}...")

### 提示优化技巧

In [ ]:

def prompt_optimization_techniques():
    """提示优化技巧"""
    print("\n" + "=" * 50)
    print("提示优化技巧")
    print("=" * 50)

    # 对比不同提示效果
    question = "如何提高Python代码性能？"

    # 基础提示
    basic_prompt = PromptTemplate.from_template("{question}")

    # 优化提示1：添加上下文
    context_prompt = PromptTemplate.from_template("""
作为Python性能优化专家，请回答：{question}
请提供具体的优化技巧和代码示例。
""")

    # 优化提示2：结构化要求
    structured_prompt = PromptTemplate.from_template("""
请回答以下Python性能问题：{question}

请按照以下结构回答：
1. 性能瓶颈识别方法
2. 常见优化技巧（至少5个）
3. 具体代码示例
4. 性能测试方法
5. 最佳实践建议

每个部分请提供详细说明：
""")

    # 优化提示3：示例驱动
    example_driven_prompt = PromptTemplate.from_template("""
{question}

参考以下优化示例的格式：

示例：列表推导式优化
问题代码：
```python
result = []
for i in range(1000):
    if i % 2 == 0:
        result.append(i * 2)
```

优化代码：
```python
result = [i * 2 for i in range(1000) if i % 2 == 0]
```

性能提升：约30%

请提供类似格式的5个优化技巧：
""")

    prompts = [
        ("基础提示", basic_prompt),
        ("添加上下文", context_prompt),
        ("结构化要求", structured_prompt),
        ("示例驱动", example_driven_prompt)
    ]

    for name, prompt in prompts:
        print(f"\n{name}:")
        response = llm.invoke(prompt.format(question=question))
        print(f"回答长度: {len(response)} 字符")
        print(f"回答预览: {response[:150]}...")



## 总结

这个完整的提示词教程涵盖了：
1. 基础提示模板 - 字符串和聊天模板的基本用法
2. 少样本学习 - 通过示例引导模型输出
3. 管道提示 - 组合多个提示组件
4. 条件和动态提示 - 根据输入动态生成提示
5. 提示组合和复用 - 构建可复用的提示组件
6. 高级技巧 - 结构化输出、思维链、角色扮演等